<a href="https://colab.research.google.com/github/Jin-jin-jara/Deep-Learning/blob/master/keras_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_iris

TensorFlow 2.x selected.


In [2]:
x, y = load_iris(return_X_y=True)
x.shape, y.shape, set(y)   # 150개의 데이터, 클래스 k = 3

((150, 4), (150,), {0, 1, 2})

In [0]:
class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.optimizer = tf.optimizers.SGD(learning_rate=0.01)   # stochastic gradient decent 확률적(배치로 나눠서 하기 때문에 stochastic이란 이름이 붙었다) 경사 하강
    self.dense = keras.layers.Dense(units=3, activation=keras.activations.softmax)  # 클래스가 3개니까 unit은 3이다.

  def call(self, x):
    h = self.dense(x)
    # h = h[:,0] # y가 1차원이기 떄문에 차원 축소를 해준다 2d -> 1d 멀티 클래스라 원핫인코딩을 해줘야해서 y는 2차원이 된다 하지 말자
    return h

  def get_loss(self, y, h):
    # 분류이기 때문에 crossentropy 공식을 쓴다
    cross_entropy = -(y * tf.math.log(h) + (1-y) * tf.math.log(1-h))
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):
    # h 3개의 확률 : (m, 3), y : (m)
    predict = tf.argmax(h, -1)
    # tf.cast 는 bool 형식의 값들을 0과 1로 만든다.
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32))   # bool 값이 뜬다  True -> 1, False -> 0

  def train(self, x, y, epoch=1):
    # x : (m, 4) 2d, y : (m) 1d
    y_hot = tf.one_hot(y, depth=3,)  # 2d로 변경  (m, 3)
    # y_hot = tf.reshape(-1, 3)
    for i in range(epoch):
      with tf.GradientTape() as tape: # 경사 기록 장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)
    grads = tape.gradient(loss, self.trainable_variables)  # 경사 계산
    self.optimizer.apply_gradients(zip(grads, self.trainable_variables))   # 가중치에서 경사를 빼서 저장
    self.get_accuracy(y, h)
    print("%d/%d loss:%.3f acc:%.3f"%(i, epoch, loss, self.acc))

model = MyModel()

In [0]:
class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.opt = tf.optimizers.Adam(learning_rate=0.005)   # stochastic gradient decent 확률적(배치로 나눠서 하기 때문에 stochastic이란 이름이 붙었다) 경사 하강
    self.conv1 = keras.layers.Conv2D(128,3, padding = 'same', input_shape=(8,8,1), activation=tf.nn.elu)
    self.pooling1 = keras.layers.MaxPool2D( padding = 'same')
    self.conv2 = keras.layers.Conv2D(64,3, padding='same', activation=tf.nn.elu)
    self.pooling2 = keras.layers.MaxPool2D( padding = 'same')
    self.flatten = keras.layers.Flatten()
    self.dense1 = keras.layers.Dense(10, activation=keras.activations.softmax)
    # self.dense2 = keras.layers.Dense(10, activation=keras.activations.relu)
    

  def call(self, x):
    x = tf.reshape(x, [-1, 8, 8, 1])
    x = self.conv1(x)
    x = self.pooling1(x)
    x = self.conv2(x)
    x = self.pooling2(x)
    x = self.flatten(x)
    # x = self.dense2(x)
    h = self.dense1(x)
    return h
    # h = h[:,0] # y가 1차원이기 떄문에 차원 축소를 해준다 2d -> 1d 멀티 클래스라 원핫인코딩을 해줘야해서 y는 2차원이 된다 하지 말자
    

  def get_loss(self, y, h):
    # 분류이기 때문에 crossentropy 공식을 쓴다
    h = tf.clip_by_value(h, 1e-8, 1 - 1e-8)
    cross_entropy = -(y * tf.math.log(h) + (1-y) * tf.math.log(1-h))
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):
    # h 10개의 확률 : (m, 10), y : (m)
    predict = tf.argmax(h, -1)
    # tf.cast 는 bool 형식의 값들을 0과 1로 만든다.
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32))   # bool 값이 뜬다  True -> 1, False -> 0

  def train(self, x, y, epoch=1):
    # x : (m, 4) 2d, y : (m) 1d
    y_hot = tf.one_hot(y, depth=10)  # 2d로 변경  (m, 3)
    # y_hot = tf.reshape(-1, 3)
    for i in range(epoch):
      with tf.GradientTape() as tape: # 경사 기록 장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)
      
      grads = tape.gradient(loss, self.trainable_variables)  # 경사 계산
      self.opt.apply_gradients(zip(grads, self.trainable_variables))   # 가중치에서 경사를 빼서 저장
      self.get_accuracy(y, h)
      print("%d/%d loss:%.3f acc:%.3f"%(i, epoch, loss, self.acc))

model = MyModel()

In [5]:
from sklearn.datasets import load_digits

x, y = load_digits(return_X_y=True)

# x = np.reshape(x, (1797, 8, 8, 1))
x.shape, y.shape

((1797, 64), (1797,))

In [10]:
x.shape, y.shape

((1797, 64), (1797,))

In [11]:
model.train(x, y, 100)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0/100 loss:0.409 acc:0.126
1/100 loss:1.054 acc:0.098
2/100 loss:1.326 acc:0.101
3/100 loss:1.464 acc:0.100
4/100 loss:1.201 acc:0.107
5/100 loss:0.945 acc:0.147
6/100 loss:0.922 acc:0.157
7/100 loss:0.748 acc:0.196
8/100 loss:0.536 acc:0.354
9/100 loss:0.363 acc:0.301
10/100 loss:0.447 acc:0.205
11/100 loss:0.440 acc:0.265
12/100 loss:0.275 acc:0.408
13/100 loss:0.241 acc:0.496
14/100 loss:0.237 acc:0.491
15/100 loss:0.203 acc:0.611
16/100 loss:0.178 acc:0.687
17/100 loss:0.164 acc:0.705
18/100 loss:0.

In [0]:
# 50/100 loss:0.021 acc:0.974
# 50/100 loss:0.012 acc:0.986